In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy.linalg as LA
import random
import pandas as pd
from numpy import inf
import torch
from torch import nn
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch.optim as optim
import json
import itertools
import math
import pyprind
from random import sample

In [34]:
print(torch.__version__)

1.13.1


In [2]:
#f = open('data_irl_model/Feb23_data_exploratory_policy.json')
f = open('../data_irl_model/Feb22_dataSVO.json')
data_irl = json.load(f)

In [3]:
data_keys=list(data_irl.keys())
data_keys.sort()

keys_for_training=[]
keys_for_testing=[]

index=0
for vehicle in data_keys:
    if index%10==0:
        keys_for_testing.append(vehicle)
        index+=1
    else:
        keys_for_training.append(vehicle)
        index+=1

In [4]:
len(keys_for_training)

5400

In [ ]:
data_irl[str(keys_for_testing[10])]

In [5]:
key=keys_for_training[967]
key=str(key)                
############# Create the data for the training locally- we are restricting the number of timesteps to num_timesteps######
negative_vals=[]
for i in [0,1]:
    negative_vals.append(list(filter(lambda x: x < 1, data_irl[key][i])))
num_timesteps=min(20,len(negative_vals[0]),len(negative_vals[1]))

print("0: ", negative_vals[0])
print("1: ", negative_vals[1])

print("num_timesteps: ", num_timesteps)

0:  [-50.0, -48.1, -46.4, -44.75796913937893, -43.040772632427355, -41.271099841265446, -39.4745449925036, -37.67293664052113, -35.88498649497113, -34.127188049008055, -32.415130406928334, -30.764126999474023, -29.19012274020666, -27.71120188485676, -26.348038955235424, -25.125573218590144, -24.074698426038893, -23.218097074828766, -22.561495723618638, -22.10489437240851, -21.848293021198383, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.76999234559332, -21.73049042434515, -21.59148658184881, -21.332482739352468, -20.95347889685613, -20.45447505435979, -19.83547121186345, -19.09646736936711, -18.237463526870773, -17.258459684374436, -16.1594558418781, -14.940451999381757, -13.601448156885418, -12.142444314389081, -10.56344047189274, -8.864436629396401,

In [ ]:
index_before_zero=data_irl[key][1].index(negative_vals[1][len(negative_vals[1])-1])
print("index_before_zero: ", index_before_zero)
num_timesteps=20
indices_to_train=[]
if index_before_zero-num_timesteps>=0:
    indices_to_train=range(index_before_zero-num_timesteps,index_before_zero+2)
else:
    indices_to_train=range(0,index_before_zero+2)

for ind in indices_to_train:
    print(data_irl[key][0][ind])
for ind in indices_to_train:
    print(data_irl[key][1][ind])

In [4]:
#########################################################################
################################ AIS_gen ################################

################################ Model 1 ################################ 

class GenerateAis1(nn.Module):

    #def __init__(self,n_input,n_state,n_psi2_in=64,n_psi2_out=128):
    def __init__(self,n_input,n_state,n_psi2_in=8,n_psi2_out=16):
        super(GenerateAis1,self).__init__()
        self.PSI_layer1=nn.Linear(n_input,n_psi2_in)     # Use RELU after
        self.PSI_layer2=nn.Linear(n_psi2_in,n_psi2_out)      # Use RELU after
        self.PSI_layer3=nn.GRUCell(n_psi2_out,n_state)       # This is the Gated layer

    def forward(self,x,h):
        #x=torch.transpose(x,0,1)
        #h=torch.transpose(h,0,1)
        x=torch.relu(self.PSI_layer1(x))
        x=torch.relu(self.PSI_layer2(x))
        h=self.PSI_layer3(x,h)
        return h     


In [5]:
#########################################################################
################################ AIS_pred ###############################

################################ Model 1 ################################ 

class PredictAis1(nn.Module):
    
    #def __init__(self,n_output,n_state,n_phi2_in=16,n_phi2_out=8):
    def __init__(self,n_output,n_state,n_phi2_in=6,n_phi2_out=8):
        super(PredictAis1,self).__init__()
        self.PHI_layer1=nn.Linear(n_state,n_phi2_in)  # Use RELU after
        self.PHI_layer2=nn.Linear(n_phi2_in,n_phi2_out)     # Use RELU after
        self.PHI_layer3=nn.Linear(n_phi2_out,n_output)         # mean vector of a unit-variance multivariate Gaussian distribution, samples from which are used to predict the next observation

    def forward(self,x):
        x=torch.relu(self.PHI_layer1(x))
        x=torch.relu(self.PHI_layer2(x))
        output=self.PHI_layer3(x)
        return output


In [7]:
negative_vals=[]
for i in [0,1]:
    negative_vals.append(list(filter(lambda x: x < 1, data_irl[str(keys_for_testing[0])][i])))
print(negative_vals)

[[-50.0, -48.1, -46.4, -44.761122658207675, -43.05127992142175, -41.28274312577878, -39.46536936838444, -37.60696288521823, -35.71356531742613, -33.78968838023041, -31.838501427503736, -29.861986078108643, -27.861070285758846, -25.835754674634355, -23.785244071125202, -21.708096136013104, -19.60239595060883, -17.4659597386202, -15.29656284320157, -13.092178062307937, -10.851202938627893, -8.573166165623306, -6.259465148147179, -3.913311880225578, -1.5399957277141993, 0.8524037569850123], [-56.30088862147648, -54.726865665496135, -53.206261143616395, -51.740175800649745, -50.32985055901144, -48.97686996364336, -47.68307154610725, -46.450560206681246, -45.281725232740015, -44.17925946401058, -43.1461798219091, -42.18584795159752, -41.30198901475314, -40.49870563862401, -39.7804825945506, -39.15217590591451, -38.61897783980735, -38.186346917018525, -37.85989037368617, -37.64518666049085, -37.54753932832985, -37.528616713070534, -37.528616713070534, -37.528616713070534, -37.528616713070534

In [27]:
class NN_structure(object):
    def __init__(self,n_epochs,min_timesteps,n_rollout,n_skips_per_rollout,n_test,n_input,n_output,n_state_enc,learning_rate,ais_gen_model,ais_pred_model,device):
        self.n_epochs=n_epochs
        self.min_timesteps=min_timesteps
        self.rollout=n_rollout
        self.n_skips=n_skips_per_rollout
        self.n_test=n_test
        self.n_input=n_input
        self.n_output=n_output
        self.n_state_enc=n_state_enc    # Hidden state size in RNN
        self.learning_rate=learning_rate                
        self.device=device
        self.ais_gen_model=ais_gen_model
        self.ais_pred_model=ais_pred_model

        self.n_psi2_in=8
        self.n_psi2_out=16
        self.n_phi2_in=2
        self.n_phi2_out=4
        self.n_psi1_out=8
        
        if ais_gen_model==1:
            self.gen_model=GenerateAis1(self.n_input,self.n_state_enc).to(self.device)      
        
        #if ais_gen_model==2: 
            #self.gen_model=GenerateAis2(self.n_input,self.n_state_enc,self.n_psi1_out).to(self.device)        
            #self.gen_model=GenerateAis2_LSTM(self.n_input,self.n_state_enc,self.n_psi1_out).to(self.device)
            #self.gen_model=GenerateAis_LSTMShallow(self.n_input,self.n_state_enc).to(self.device)  

        if ais_pred_model==1:
            self.pred_model=PredictAis1(self.n_output,self.n_state_enc).to(self.device)
        
        #if ais_pred_model==2:
            #self.pred_model=PredictAis2(self.n_output,self.n_state_enc+1,self.n_state_dec).to(self.device)
            #self.pred_model=PredictAis2_LSTM(self.n_output,self.n_state_enc+1,self.n_state_dec).to(self.device)
            #self.pred_model=PredictAis_LSTMShallow(self.n_state_enc+1,self.n_state_dec).to(self.device)

    def save_model_weights(self,text="(:*_*:)"):
        #name_gen_path="trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
        #torch.save(self.gen_model.state_dict(),name_gen_path)
        
        name_gen_path="_"
        name_pred_path="_"
        if self.ais_pred_model==1:
            name_pred_path="../trained_models/IRL_based_models/IRL_SVOpolicy_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
            name_gen_path="../trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"

        if self.ais_pred_model==2:
            name_pred_path="../trained_models/IRL_based_models/IRL_SVOpolicy_multi_rnn_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"
            name_gen_path="../trained_models/IRL_based_models/IRL_SVOpolicy_multi_rnn_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"

        torch.save(self.gen_model.state_dict(),name_gen_path)
        torch.save(self.pred_model.state_dict(),name_pred_path)

        return "saved"
    
    def load_model_weights(self,text="(:*_*:)"):
        #name_gen_path="trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
        #torch.save(self.gen_model.state_dict(),name_gen_path)
        #self.gen_model.load_state_dict(torch.load(name_gen_path))
        #self.gen_model.eval()

        name_gen_path="_"
        name_pred_path="_"
        
        if self.ais_pred_model==1:
            name_pred_path="../trained_models/IRL_based_models/IRL_SVOpolicy_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"
            name_gen_path="../trained_models/IRL_based_models/IRL_SVOpolicy_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+text+".pth"

        if self.ais_pred_model==2:
            name_pred_path="../trained_models/IRL_based_models/IRL_SVOpolicy_multi_rnn_ais_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"
            name_gen_path="../trained_models/IRL_based_models/IRL_SVOpolicy_multi_rnn_ais_gen"+str(self.ais_gen_model)+"_"+str(self.n_psi2_in)+str(self.n_psi2_out)+"_pred"+str(self.ais_pred_model)+"_"+str(self.n_phi2_in)+str(self.n_phi2_out)+"_epochs"+str(self.n_epochs)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state_enc)+"_"+str(self.n_state_dec)+text+".pth"
        
        #torch.save(self.pred_model.state_dict(),name_pred_path)
        self.gen_model.load_state_dict(torch.load(name_gen_path))
        self.gen_model.eval()

        self.pred_model.load_state_dict(torch.load(name_pred_path))
        self.pred_model.eval()

        return "loaded"
    

    def train(self):
        train_bar=pyprind.ProgBar(self.n_epochs)
        self.optimizer = torch.optim.Adam(list(self.gen_model.parameters()) + list(self.pred_model.parameters()), lr=self.learning_rate, amsgrad=True)

        time_step_loss=0
        for epoch in range(self.n_epochs):

            train_bar.update()
            keys_training=sample(keys_for_training,len(keys_for_training)) #shuffle the keys
            #train_bar=pyprind.ProgBar(len(keys_for_training))
            epoch_loss=0
            for key in keys_training:
                #for key in keys_for_testing[0]:
                #train_bar.update()
                key=str(key) 
                #print(key)                               
                ############# Create the data for the training locally- we are restricting the number of timesteps to num_timesteps ######
                negative_vals=[]
                for i in [0,1]:
                    negative_vals.append(list(filter(lambda x: x < 1, data_irl[key][i])))
                num_timesteps=min(self.min_timesteps,len(negative_vals[0]),len(negative_vals[1]))
                index_before_zero=data_irl[key][1].index(negative_vals[1][len(negative_vals[1])-1])
                #print("index_before_zero: ", index_before_zero)
                #num_timesteps=20
                indices_to_train=[]
                if index_before_zero-num_timesteps>=0:
                    indices_to_train=range(index_before_zero-num_timesteps,index_before_zero+1)
                else:
                    indices_to_train=range(0,index_before_zero+2)
                #print("ndices_to_train: ", indices_to_train)
                #######################################

                #### IMPORTANT Cropping the indices as per availability of data - 
                # What if index_before_zero == length of the list and we are looking 
                # to predict data that is in existent

                '''
                ## No skip between predictions
                indices_to_train_cropped=[]
                if index_before_zero in [len(data_irl[key][1])-1,len(data_irl[key][1])-2,len(data_irl[key][1])-3]:
                    indices_to_train_cropped=indices_to_train[0:-1*self.rollout]
                else:
                    indices_to_train_cropped=indices_to_train
                '''
                
                ## Use this when we want to skip between predictions
                ## Skip indicates the predicitons are for t+n_skips,t+2*n_skips,t+3*n_skips,...
                indices_to_train_cropped=[]
                if index_before_zero in range(len(data_irl[key][1])-1,len(data_irl[key][1])-self.rollout*(self.n_skips+1),-1):
                    indices_to_train_cropped=indices_to_train[0:-self.rollout*(self.n_skips+1)]
                else:
                    indices_to_train_cropped=indices_to_train

                # Ais initialization
                ais_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device)
                state_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device) # Only if we use a LSTM
                action_list=[0,0] ## initialize the action list differently - but we can try giving it the actual values to iterate                

                time_step_loss=0

                # Initialize the origin offset only for position
                #origin_offset_CAV=data_irl[key][0][indices_to_train[0]]
                #origin_offset_HDV=data_irl[key][1][indices_to_train[0]]
                origin_offset_CAV=0    
                origin_offset_HDV=0

                for time_step in indices_to_train_cropped:

                    observation_list=[]
                    for i in range(len(data_irl[key])-2): # exclude the last two columns (actions/inputs)
                        temp=data_irl[key][i][time_step]
                        if i==0:
                            temp=temp-origin_offset_CAV
                        elif i==1:
                            temp=temp-origin_offset_HDV
                        if temp!= None:
                            observation_list.append(temp)
                    input_list=[]
                    input_list.extend(observation_list)
                    input_list.extend(action_list)
                    input_tensor=torch.tensor(input_list,dtype=torch.float).to(self.device)                    

                    # Update action list for the next time step 
                    for i in [4,5]:
                        temp=data_irl[key][i][time_step]
                        if temp!= None:
                            action_list[i-4]=temp
                        else:
                            action_list[i-4]=0

                    '''
                    ### For no skip between predictions
                    reference_list=[]
                    for k in range(self.rollout):
                        for i in [1,3,5]:#range(len(data_irl[key])):
                            if i==1:# min is self.rollout
                                #rint("key",key)
                                #print("i",i)
                                #print("time_step+k+1",time_step+k+1)
                                reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                            
                            elif i==3:
                                reference_list.append(data_irl[key][i][time_step+k+1])

                            elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                reference_list.append(data_irl[key][i][time_step+k])
                            
                            else:
                                reference_list.append(0) # Zero padding for the input action

                    '''

                    
                    ### Skip between predictions
                    reference_list=[]
                    if (time_step+self.rollout*(self.n_skips+1))>=len(data_irl[key][i]):
                        continue
                    else:
                        for k in range(self.n_skips,self.rollout*(self.n_skips+1),self.n_skips+1):
                            for i in [1,3,5]:#range(len(data_irl[key])):
                                if i==1:# min is self.rollout
                                    #rint("key",key)
                                    #print("i",i)
                                    #print("time_step+k+1",time_step+k+1)
                                    reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                                
                                elif i==3:
                                    reference_list.append(data_irl[key][i][time_step+k+1])

                                elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                    reference_list.append(data_irl[key][i][time_step+k])
                                
                                else:
                                    reference_list.append(0) # Zero padding for the input action

                                           

                    '''
                    # This is to be as 3* as long as the number of rollouts - 3 for each rollout
                    internal_rollout=min(self.rollout,len(data_irl[key][0])-time_step-2)
                    

                    reference_list=[]

                    if internal_rollout==self.rollout:
                        for k in range(self.rollout):
                            for i in [1,3,5]:#range(len(data_irl[key])):
                                if i==1:# min is self.rollout
                                    reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                                
                                elif i==3:
                                    reference_list.append(data_irl[key][i][time_step+k+1])

                                elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                    reference_list.append(data_irl[key][i][time_step+k])
                                
                                else:
                                    reference_list.append(0) # Zero padding for the input action
                            
                    else:
                        for k in range(internal_rollout):
                            for i in [1,3,5]:#range(len(data_irl[key])):
                                if i==1:# min is not self.rollout
                                    reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                                
                                elif i==3:
                                    reference_list.append(data_irl[key][i][time_step+k+1])

                                elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                    reference_list.append(data_irl[key][i][time_step+k])
                                
                                else:
                                    reference_list.append(0) # Zero padding for the input action
                                    
                        for k in range(internal_rollout,self.rollout):
                            for i in [1,3,5]:
                                reference_list.append(0)
                    '''


                    reference_tensor=torch.tensor(reference_list,dtype=torch.float).to(self.device)

                    ais_rollout_enc=self.gen_model(input_tensor,ais_rollout_enc)
                    decoder_prediction=self.pred_model(ais_rollout_enc)

                    ## Mulitvariate normal distribution loss
                    #time_step_loss+=torch.distributions.MultivariateNormal(decoder_prediction,torch.eye(self.n_output).to(self.device)).log_prob(reference_tensor)
                    ## MSE loss
                    time_step_loss+=torch.nn.functional.mse_loss(decoder_prediction,reference_tensor)                    
                    #print("time_step_loss: ",time_step_loss)
                    #print("reference_tensor: ",reference_tensor)
                    #print("decoder_prediction: ",decoder_prediction,"\n")
                    
                    
               
                epoch_loss+=time_step_loss
                self.optimizer.zero_grad()
                time_step_loss.backward()
                self.optimizer.step()
                
                ## Printing the gradients
                '''
                print("Printing the gradients-encoder")
                print(self.gen_model.PSI_layer3.weight_ih.grad)
                print(self.gen_model.PSI_layer3.weight_hh.grad)
                print(self.gen_model.PSI_layer3.bias_ih.grad)
                print(self.gen_model.PSI_layer3.bias_hh.grad)

                print("Printing the gradients-decoder")                
                print(self.pred_model.PHI_layer1.weight.grad)
                print(self.pred_model.PHI_layer2.weight.grad)
                print(self.pred_model.PHI_layer3.weight.grad)           
                '''
                
            print("epoch_loss is ",epoch_loss,"\n")                
            print("saving model weights for epoch ",epoch)
            status=self.save_model_weights("encoder_rnn_decoder_simple_nn_n_rollout_3")

    def test(self,test_key):
        test_results=[]
        time_step_loss=0
        keys_testing=keys_for_testing[test_key]# sample(keys_for_training,len(keys_for_training)) #shuffle the keys
        #train_bar=pyprind.ProgBar(len(keys_for_training))
        epoch_loss=0
        for key in keys_testing:
            #for key in keys_for_testing[0]:
            #train_bar.update()
            key=str(key) 
            #print(key)                               
            ############# Create the data for the training locally- we are restricting the number of timesteps to num_timesteps ######
            negative_vals=[]
            for i in [0,1]:
                negative_vals.append(list(filter(lambda x: x < 1, data_irl[key][i])))
            num_timesteps=min(self.min_timesteps,len(negative_vals[0]),len(negative_vals[1]))
            index_before_zero=data_irl[key][1].index(negative_vals[1][len(negative_vals[1])-1])
            #print("index_before_zero: ", index_before_zero)
            #num_timesteps=20
            indices_to_test=[]
            if index_before_zero-num_timesteps>=0:
                indices_to_test=range(index_before_zero-num_timesteps,index_before_zero+1)
            else:
                indices_to_test=range(0,index_before_zero+2)
            #print("ndices_to_train: ", indices_to_train)
            #######################################

            #### IMPORTANT Cropping the indices as per availability of data - 
            # What if index_before_zero == length of the list and we are looking 
            # to predict data that is in existent

            '''
            ## No skip between predictions
            indices_to_train_cropped=[]
            if index_before_zero in [len(data_irl[key][1])-1,len(data_irl[key][1])-2,len(data_irl[key][1])-3]:
                indices_to_train_cropped=indices_to_train[0:-1*self.rollout]
            else:
                indices_to_train_cropped=indices_to_train
            '''
            
            ## Use this when we want to skip between predictions
            ## Skip indicates the predicitons are for t+n_skips,t+2*n_skips,t+3*n_skips,...
            indices_to_test_cropped=[]
            if index_before_zero in range(len(data_irl[key][1])-1,len(data_irl[key][1])-self.rollout*(self.n_skips+1),-1):
                indices_to_test_cropped=indices_to_test[0:-self.rollout*(self.n_skips+1)]
            else:
                indices_to_test_cropped=indices_to_test

            # Ais initialization
            ais_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device)
            state_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device) # Only if we use a LSTM
            action_list=[0,0] ## initialize the action list differently - but we can try giving it the actual values to iterate                

            time_step_loss=0

            # Initialize the origin offset only for position
            #origin_offset_CAV=data_irl[key][0][indices_to_train[0]]
            #origin_offset_HDV=data_irl[key][1][indices_to_train[0]]
            
            # For now we are not doing origin offset 
            origin_offset_CAV=0    
            origin_offset_HDV=0

            for time_step in indices_to_test_cropped:

                observation_list=[]
                for i in range(len(data_irl[key])-2): # exclude the last two columns (actions/inputs)
                    temp=data_irl[key][i][time_step]
                    if i==0:
                        temp=temp-origin_offset_CAV
                    elif i==1:
                        temp=temp-origin_offset_HDV
                    if temp!= None:
                        observation_list.append(temp)
                input_list=[]
                input_list.extend(observation_list)
                input_list.extend(action_list)
                input_tensor=torch.tensor(input_list,dtype=torch.float).to(self.device)                    

                # Update action list for the next time step 
                for i in [4,5]:
                    temp=data_irl[key][i][time_step]
                    if temp!= None:
                        action_list[i-4]=temp
                    else:
                        action_list[i-4]=0

                '''
                ### For no skip between predictions
                reference_list=[]
                for k in range(self.rollout):
                    for i in [1,3,5]:#range(len(data_irl[key])):
                        if i==1:# min is self.rollout
                            #rint("key",key)
                            #print("i",i)
                            #print("time_step+k+1",time_step+k+1)
                            reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                        
                        elif i==3:
                            reference_list.append(data_irl[key][i][time_step+k+1])

                        elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                            reference_list.append(data_irl[key][i][time_step+k])
                        
                        else:
                            reference_list.append(0) # Zero padding for the input action

                '''

                
                ### Skip between predictions
                reference_list=[]
                if (time_step+self.rollout*(self.n_skips+1))>=len(data_irl[key][i]):
                    continue
                else:
                    for k in range(self.n_skips,self.rollout*(self.n_skips+1),self.n_skips+1):
                        #print(range(self.n_skips,self.rollout*(self.n_skips+1),self.n_skips+1))
                        #print("k",k,"\n")
                        for i in [1,3,5]:#range(len(data_irl[key])):
                            if i==1:# min is self.rollout
                                #rint("key",key)
                                #print("i",i)
                                #print("time_step+k+1",time_step+k+1)
                                reference_list.append(data_irl[key][i][time_step+k+1]-origin_offset_HDV)
                            
                            elif i==3:
                                reference_list.append(data_irl[key][i][time_step+k+1])

                            elif i>=4 and data_irl[key][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                                reference_list.append(data_irl[key][i][time_step+k])
                            
                            else:
                                reference_list.append(0) # Zero padding for the input action


                reference_tensor=torch.tensor(reference_list,dtype=torch.float).to(self.device)
                
                temp=[]
                temp.append(reference_tensor.detach().cpu().numpy())

                ais_rollout_enc=self.gen_model(input_tensor,ais_rollout_enc)
                decoder_prediction=self.pred_model(ais_rollout_enc)

                ## Mulitvariate normal distribution loss
                #time_step_loss+=torch.distributions.MultivariateNormal(decoder_prediction,torch.eye(self.n_output).to(self.device)).log_prob(reference_tensor)
                ## MSE loss
                time_step_loss+=torch.nn.functional.mse_loss(decoder_prediction,reference_tensor)                    
                #print("time_step_loss: ",time_step_loss)
                print("time_step: ",time_step)
                print("reference_tensor: ",reference_tensor)
                print("decoder_prediction: ",decoder_prediction,"\n")

                temp.append(decoder_prediction.detach().cpu().numpy())
                test_results.append(temp)
            
        return test_results



In [24]:
####### Parameters #######
n_epochs=150
min_timesteps=30
n_rollout=3
n_skips_per_rollout=1
n_test=1
n_input=6
n_output=n_rollout*3 # 3 for each of the next n_rollout time steps
n_state_enc=4
learning_rate=0.0003
gen_model=1
pred_model=1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
####### Initialise the neural network #######

network=NN_structure(n_epochs,min_timesteps,n_rollout,n_skips_per_rollout,n_test,n_input,n_output,n_state_enc,learning_rate,gen_model,pred_model,device)
print("Training has started")
network.train()


Training has started
epoch_loss is  tensor(6794951.5000, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  0
epoch_loss is  tensor(305949.0938, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  1
epoch_loss is  tensor(168722.3594, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  2
epoch_loss is  tensor(119809.2656, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  3


0% [#                             ] 100% | ETA: 08:42:18

epoch_loss is  tensor(94028.3125, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  4
epoch_loss is  tensor(85959.7188, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  5
epoch_loss is  tensor(70665.7109, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  6
epoch_loss is  tensor(55459.9844, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  7


0% [##                            ] 100% | ETA: 09:05:12

epoch_loss is  tensor(51536.4883, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  8
epoch_loss is  tensor(50609.4648, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  9
epoch_loss is  tensor(56143.7148, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  10
epoch_loss is  tensor(57706.1172, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  11
epoch_loss is  tensor(42726.9023, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  12


...

epoch_loss is  tensor(13093.1816, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  143
epoch_loss is  tensor(13028.3760, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  144
epoch_loss is  tensor(13054.7070, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  145
epoch_loss is  tensor(13091.4111, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  146
epoch_loss is  tensor(13008.3379, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  147


0% [##############################] 100% | ETA: 00:00:00

epoch_loss is  tensor(13346.0127, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  148



Total time elapsed: 10:54:15


epoch_loss is  tensor(13094.1250, device='cuda:0', grad_fn=<AddBackward0>) 

saving model weights for epoch  149


In [ ]:
################ Sorting issues with indexing ##########################
#key= 2825
#i= 1
#time_step+k+1 =201

#data_irl[str(2825)][1][200]

key=str(1289)

############# Create the data for the training locally- we are restricting the number of timesteps to num_timesteps ######
negative_vals=[]
for i in [0,1]:
    negative_vals.append(list(filter(lambda x: x < 1, data_irl[key][i])))
num_timesteps=min(15,len(negative_vals[0]),len(negative_vals[1]))
index_before_zero=data_irl[key][1].index(negative_vals[1][len(negative_vals[1])-1])
#print("index_before_zero: ", index_before_zero)
num_timesteps=20
indices_to_train=[]
if index_before_zero-num_timesteps>=0:
    indices_to_train=range(index_before_zero-num_timesteps,index_before_zero+1)
    print("ndices_to_train: if ", indices_to_train)
else:
    indices_to_train=range(0,index_before_zero+2)
    print("ndices_to_train: else ", indices_to_train)
#######################################
print("values and index near zero",index_before_zero,len(data_irl[key][1]))
#print(data_irl[key][1][index_before_zero-1])
#print(data_irl[key][1][index_before_zero])
#print(data_irl[key][1][index_before_zero+1])

#### IMPORTANT Cropping the indices as per availability of data - 
# What if index_before_zero == length of the list and we are looking 
# to predict data that is in existent

n_rollout=4
n_skip_per_rollout=2
indices_to_train_cropped=[]
if index_before_zero in range(len(data_irl[key][1])-1,len(data_irl[key][1])-n_rollout*(n_skip_per_rollout+1),-1):
    #[len(data_irl[key][1])-1,len(data_irl[key][1])-2,len(data_irl[key][1])-3]
    #indices_to_train_cropped=indices_to_train[0:-n_rollout]
    indices_to_train_cropped=indices_to_train[0:-n_rollout*(n_skip_per_rollout+1)]
else:
    indices_to_train_cropped=indices_to_train
print("indices_to_train_cropped",indices_to_train_cropped)
print("length of array",len(data_irl[key][i]))
print("min(n_rollout*(n_skip_per_rollout+1),len(data_irl[key][i]))",min(n_rollout*(n_skip_per_rollout+1),len(data_irl[key][i])))

for time_step in indices_to_train_cropped:
    reference_list=[]
    print("timestep_where skip began",time_step)
    if (time_step+n_rollout*(n_skip_per_rollout+1))>=len(data_irl[key][i]):
        continue
    else:
        for k in range(n_skip_per_rollout,n_rollout*(n_skip_per_rollout+1),n_skip_per_rollout+1):
            i=1# min is self.rollout
            print("key",key)
            print("i",i)
            print("time_step+k+1",time_step+k+1)
            reference_list.append(data_irl[key][i][time_step+k+1])
            print(reference_list)

In [77]:
a12=range(169, 190)
n_rollout=4
n_skip_per_rollout=2
a12[0:-n_rollout*(n_skip_per_rollout+1)]

range(169, 178)

In [32]:
# Test the network

####### Load the neural network weights from saved models #######

loaded_network=NN_structure(n_epochs,min_timesteps,n_rollout,n_skips_per_rollout,n_test,n_input,n_output,n_state_enc,learning_rate,gen_model,pred_model,device)
loaded_network.load_model_weights("encoder_rnn_decoder_simple_nn_n_rollout_3")

#test_keys=[keys_for_testing[15]]
test_results=loaded_network.test(503)



time_step:  7
reference_tensor:  tensor([-4.1345e+01,  7.6099e+00, -1.9274e-02, -3.8303e+01,  7.6018e+00,
        -2.0753e-02, -3.5264e+01,  7.5930e+00, -2.2337e-02], device='cuda:0')
decoder_prediction:  tensor([-41.2701,   8.0067,   0.2849, -38.0412,   8.1030,   0.1716, -34.7890,
          8.1742,   0.1147], device='cuda:0', grad_fn=<AddBackward0>) 

time_step:  8
reference_tensor:  tensor([-3.9823e+01,  7.6059e+00, -1.9998e-02, -3.6783e+01,  7.5975e+00,
        -2.1535e-02, -3.3746e+01,  7.5884e+00, -2.3151e-02], device='cuda:0')
decoder_prediction:  tensor([-3.9938e+01,  7.7452e+00,  1.3604e-01, -3.6811e+01,  7.7997e+00,
         5.5785e-02, -3.3690e+01,  7.8410e+00,  2.5527e-02], device='cuda:0',
       grad_fn=<AddBackward0>) 

time_step:  9
reference_tensor:  tensor([-3.8303e+01,  7.6018e+00, -2.0753e-02, -3.5264e+01,  7.5930e+00,
        -2.2337e-02, -3.2228e+01,  7.5836e+00, -2.3963e-02], device='cuda:0')
decoder_prediction:  tensor([-3.8245e+01,  7.5570e+00,  5.1411e-02, -3.5

In [16]:
test_results

[[array([-24.064964  ,   4.1906233 ,   0.67194194], dtype=float32),
  array([-25.229195  ,   3.1972115 ,   0.36127877, -23.918798  ,
           3.4208672 ,   0.4649216 , -22.453152  ,   3.6634502 ,
           0.5251166 ], dtype=float32)],
 [array([-23.21346   ,   4.324399  ,   0.66887933], dtype=float32),
  array([-2.4573174e+01,  4.1392016e+00, -1.2943864e-02, -2.2882221e+01,
          4.2043829e+00,  6.4425588e-02, -2.1170528e+01,  4.3009992e+00,
          1.4079911e-01], dtype=float32)],
 [array([-22.335272 ,   4.457504 ,   0.6655236], dtype=float32),
  array([-2.4191008e+01,  4.4817510e+00,  2.3482800e-02, -2.2353559e+01,
          4.5476985e+00,  6.8805099e-02, -2.0519100e+01,  4.6405687e+00,
          1.2878224e-01], dtype=float32)],
 [array([-21.430532 ,   4.589883 ,   0.6618953], dtype=float32),
  array([-23.419863  ,   4.531012  ,   0.2751075 , -21.544128  ,
           4.6796565 ,   0.2870773 , -19.645433  ,   4.8382454 ,
           0.30386758], dtype=float32)],
 [array([-20.4

In [ ]:
## Test function ##
def test(self,test_keys):
    test_bar=pyprind.ProgBar(len(test_keys))
    self.comparision_list=[]
    #self.load_model_weights("Rollout_10")
    #print("loaded model weights")
        
    self.test_run_comparision=[]
    for test_run in range(len(test_keys)):
        test_bar.update()
        #key_test=str(test_keys[test_run])
        #key_test=str(keys_for_training[test_run])
        key_test=keys_for_training[0]
        
        ############# locally restrict the data for the training - we are restricting the number of timesteps to num_timesteps######
        negative_vals=[]
        for i in [0,1]:
            negative_vals.append(list(filter(lambda x: x < 1, data_irl[key_test][i])))

        num_timesteps=min(self.min_timesteps,len(negative_vals[0]),len(negative_vals[1]))
        
        # Ais initialization
        time_step_loss=0
        ais_rollout_enc=0.5*torch.ones(self.n_state_enc,dtype=torch.float).to(self.device)
        action_list=[0,0]             

        time_step_comparision=[]
        for time_step in range(num_timesteps):                    
            observation_list=[]
            for i in range(len(data_irl[key_test])-2): # exclude the last two columns (actions/inputs)
                temp=data_irl[key_test][i][time_step]
                if temp!= None:
                    observation_list.append(temp)
                
            input_list=[]
            input_list.extend(observation_list)
            input_list.extend(action_list)
            input_tensor=torch.tensor(input_list,dtype=torch.float).to(self.device)
            print("\n")
            print("Before rollout, the input is ",input_tensor)
            # Update action list for the next time step and for the reference
            for i in [4,5]:
                temp=data_irl[key_test][i][time_step]
                if temp!= None:
                    action_list[i-4]=temp
                else:
                    action_list[i-4]=0

            reference_list=[]
            observation_list_ref=[]
            for i in [1,3]:#range(len(data_irl[key_test])-2): # exclude the last two columns (actions/inputs)
                temp=data_irl[key_test][i][time_step+1]
                if temp!= None:
                    observation_list_ref.append(temp)                      
            reference_list.extend(observation_list_ref)
            reference_list.extend([action_list[1]])

            reference_tensor=1*torch.tensor(reference_list,dtype=torch.float).to(self.device)

            ais_rollout_dec=0.5*torch.ones(self.n_state_dec,dtype=torch.float).to(self.device) # Just initialize it to 0.5


            rollout_reference=0.5*torch.ones(self.n_output,dtype=torch.float).to(self.device)

            rollout_prediction=0.5*torch.ones(self.n_output,dtype=torch.float).to(self.device)

            #internal_rollout_horizon=min(self.rollout,len(data_irl[key_test][0])-time_step-2)
            internal_rollout_horizon=1
            rollout_loss=0

            rollout_comparison=[]
            for k in range(internal_rollout_horizon):
                temp_comparision=[]
                
                #print("\n")
                if k==0:
                    rollout_input=input_tensor
                    rollout_reference=reference_tensor
                    #ais_rollout_enc=ais
                    print("for rollout",k,"ais ",ais_rollout_enc)

                    ais_rollout_enc=self.gen_model(rollout_input,ais_rollout_enc)
                    #ais=ais_rollout_enc
                    
                    #decoder_input=list(ais_rollout_enc.detach().cpu().numpy())
                    #decoder_input.extend([time_step+k])
                    #decoder_input_tensor=torch.tensor(decoder_input,dtype=torch.float).to(self.device)
                    
                    time_step_tensor=torch.tensor([time_step+k],dtype=torch.float).to(self.device)
                    decoder_input_tensor=torch.cat((ais_rollout_enc,time_step_tensor),0)
                    rollout_prediction,ais_rollout_dec=self.pred_model(decoder_input_tensor,ais_rollout_dec)
                    
                    print("for rollout",k,"rollout_reference is ",rollout_reference)
                    print("for rollout",k,"decoder_input  is ",decoder_input_tensor,time_step+k)
                    print("for rollout",k,"rollout_prediction is ",rollout_prediction)
                    
                    rollout_loss+=-torch.distributions.MultivariateNormal(rollout_prediction,torch.eye(rollout_prediction.shape[0]).to(self.device)).log_prob(rollout_reference)

                    temp_comparision.extend(rollout_reference.detach().cpu().numpy())
                    temp_comparision.extend(rollout_prediction.detach().cpu().numpy())

                else:
                    rollout_reference_list=[]                           
                    for i in [1,3,5]:#range(len(data_irl[key_test])):
                        if i<4:
                            rollout_reference_list.append(data_irl[key_test][i][time_step+k+1])
                        elif i>=4 and data_irl[key_test][i][time_step+k]!=None: # Previous action is observed at time_step+k+1
                            rollout_reference_list.append(data_irl[key_test][i][time_step+k])
                        else:
                            rollout_reference_list.append(0) # Zero padding for the input action

                    rollout_reference=1*torch.tensor(rollout_reference_list,dtype=torch.float).to(self.device)

                    
                    #decoder_input=list(ais_rollout_enc.detach().cpu().numpy())
                    #decoder_input.extend([time_step+k])
                    #decoder_input_tensor=torch.tensor(decoder_input,dtype=torch.float).to(self.device)

                    time_step_tensor=torch.tensor([time_step+k],dtype=torch.float).to(self.device)
                    decoder_input_tensor=torch.cat((ais_rollout_enc,time_step_tensor),0)
                    rollout_prediction,ais_rollout_dec=self.pred_model(decoder_input_tensor,ais_rollout_dec)                                                
                    
                    print("for rollout",k,"rollout_reference is ",rollout_reference)
                    print("for rollout",k,"decoder_input  is ",decoder_input_tensor,time_step+k)
                    print("for rollout",k,"rollout_prediction is ",rollout_prediction)

                    temp_comparision.extend(rollout_reference.detach().cpu().numpy())
                    temp_comparision.extend(rollout_prediction.detach().cpu().numpy())
                    rollout_loss+=-torch.distributions.MultivariateNormal(rollout_prediction,torch.eye(rollout_prediction.shape[0]).to(self.device)).log_prob(rollout_reference)
                rollout_comparison.append(temp_comparision)
            time_step_comparision.append(rollout_comparison)    
        self.test_run_comparision.extend(time_step_comparision)

    return self

In [86]:
#print(len(test_results.test_run_comparision[0][1]))

#test_results.test_run_comparision[19][:]


# Plot the results only for the HDV
pos_t=[]
vel_t=[]
acc_t=[]
pos_tref=[]
vel_tref=[]
acc_tref=[]
pos_t_hdv=[]
vel_t_hdv=[]
acc_t_hdv=[]
pos_tref_hdv=[]
vel_tref_hdv=[]
acc_tref_hdv=[]

# set which timestep to plot
time_step=10

for index in range(len(test_results.test_run_comparision[time_step])):
    '''
    pos_tref.append(test_results.test_run_comparision[time_step][index][0])
    vel_tref.append(test_results.test_run_comparision[time_step][index][2])
    acc_tref.append(test_results.test_run_comparision[time_step][index][4])
    pos_t.append(test_results.test_run_comparision[time_step][index][6])
    vel_t.append(test_results.test_run_comparision[time_step][index][8])
    acc_t.append(test_results.test_run_comparision[time_step][index][10])
    
    #pos_t_hdv.append(test_results.test_run_comparision[time_step][index][1])
    #vel_t_hdv.append(test_results.test_run_comparision[time_step][index][3])
    #acc_t_hdv.append(test_results.test_run_comparision[time_step][index][5])
    #pos_tref_hdv.append(test_results.test_run_comparision[time_step][index][7])
    #vel_tref_hdv.append(test_results.test_run_comparision[time_step][index][9])
    #acc_tref_hdv.append(test_results.test_run_comparision[time_step][index][11])

    
    pos_tref_hdv.append(test_results.test_run_comparision[time_step][index][1])
    vel_tref_hdv.append(test_results.test_run_comparision[time_step][index][3])
    acc_tref_hdv.append(test_results.test_run_comparision[time_step][index][5])
    pos_t_hdv.append(test_results.test_run_comparision[time_step][index][7])
    vel_t_hdv.append(test_results.test_run_comparision[time_step][index][9])
    acc_t_hdv.append(test_results.test_run_comparision[time_step][index][11])
    '''

    pos_tref_hdv.append(test_results.test_run_comparision[time_step][index][0])
    vel_tref_hdv.append(test_results.test_run_comparision[time_step][index][1])
    acc_tref_hdv.append(test_results.test_run_comparision[time_step][index][2])
    pos_t_hdv.append(test_results.test_run_comparision[time_step][index][3])
    vel_t_hdv.append(test_results.test_run_comparision[time_step][index][4])
    acc_t_hdv.append(test_results.test_run_comparision[time_step][index][5])


IndexError: list index out of range

In [ ]:
plot_data=data_irl[str(keys_for_testing[10])]    

pos_t_hdv=[]
vel_t_hdv=[]
acc_t_hdv=[]
pos_tref_hdv=[]
vel_tref_hdv=[]
acc_tref_hdv=[]

    for index in range(len(plot_data[0])):

        pos_t_hdv.append(test_results.test_run_comparision[time_step][index][1].detach().cpu().numpy())
        vel_t_hdv.append(test_results.test_run_comparision[time_step][index][3].detach().cpu().numpy())
        acc_t_hdv.append(test_results.test_run_comparision[time_step][index][5].detach().cpu().numpy())
        pos_tref_hdv.append(test_results.test_run_comparision[time_step][index][7].detach().cpu().numpy())
        vel_tref_hdv.append(test_results.test_run_comparision[time_step][index][9].detach().cpu().numpy())
        acc_tref_hdv.append(test_results.test_run_comparision[time_step][index][11].detach().cpu().numpy())

In [29]:
# Plotting outputs #
import matplotlib.colors as mcolors
####### For X - lateral#########
fig, ax = plt.subplots()
#plt.plot(y_ref,x_ref,'r',linewidth=2.0,label="Actual")
#plt.plot(y_pred,x_pred,'b',linewidth=2.0,label="Predicted")
plt.figure(facecolor='white')
plt.plot(pos_t_hdv,mcolors.CSS4_COLORS['salmon'],linewidth=2.0,label="Actual")
plt.plot(pos_tref_hdv,mcolors.CSS4_COLORS['limegreen'],linewidth=2.0,label="Predicted")
plt.xlabel('Time in (1/10)s')
plt.ylabel('Position along the road m')
plt.title('Comparision_of_position_along_road')
plt.legend()
plt.grid()
plt.savefig('graphs/irl_graphs/Comparision_rollout10_timestep_min20_posref_pospred_g1_p2_lr0005.png')
plt.close()

fig, ax = plt.subplots()
plt.figure(facecolor='white')
plt.plot(vel_t_hdv,mcolors.CSS4_COLORS['salmon'],linewidth=2.0,label="Actual")
plt.plot(vel_tref_hdv,mcolors.CSS4_COLORS['limegreen'],linewidth=2.0,label="Predicted")
plt.xlabel('Time in (1/10)s')
plt.ylabel('Velocity along the road in m/s')
plt.title('Comparision_Lateral_Velocities')
plt.legend()
plt.grid()
plt.savefig('graphs/irl_graphs/Comparision_rollout10_timestep_min20_vref_vpre_g1_p2_lr0005.png')
plt.close()

In [78]:
test_results.test_run_comparision[1][0]
#len(test_results.test_run_comparision)

[tensor(-46.4000, device='cuda:0'),
 tensor(-53.2063, device='cuda:0'),
 tensor(8., device='cuda:0'),
 tensor(7.4680, device='cuda:0'),
 tensor(-5., device='cuda:0'),
 tensor(-1.3502, device='cuda:0'),
 tensor(-31.8437, device='cuda:0', grad_fn=<UnbindBackward0>),
 tensor(-27.8377, device='cuda:0', grad_fn=<UnbindBackward0>),
 tensor(4.1773, device='cuda:0', grad_fn=<UnbindBackward0>),
 tensor(9.0293, device='cuda:0', grad_fn=<UnbindBackward0>),
 tensor(-1.2700, device='cuda:0', grad_fn=<UnbindBackward0>),
 tensor(0.0087, device='cuda:0', grad_fn=<UnbindBackward0>)]

In [202]:
'''
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
output = torch.nn.functional.nll_loss(torch.nn.LogSoftmax(input, dim=1), target)
output.backward()
'''
m = nn.LogSoftmax(dim=1)
input = torch.randn(2, 3)
output = m(input)

input = torch.randn(3, 5, requires_grad=True)
target = torch.tensor([1, 0, 4])
output2 = torch.nn.functional.nll_loss(m(input), target)
